In [5]:
#from sklearn.model_selection import GridSearchCV
# from scikeras.wrappers import KerasClassifier
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D, Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
import tensorflow as tf
import keras_tuner as kt

fer_directory = 'data/FER2013'
ck_directory = 'data/CK+'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices()

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
# Initialise generator with rescale factor 1./255
train_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
test_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

# Preprocess training set
train_generator = train_gen.flow_from_directory(
    fer_directory + '/train',
    target_size=(48,48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

# Preprocess test set
test_generator = test_gen.flow_from_directory(
    fer_directory + '/test',
    target_size=(48,48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [8]:
def create_model(hp):
    # Define model
    model = Sequential()

    # 1st convolution layer
    model.add(Conv2D(64, (3,3), padding='same', input_shape=(48,48,1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    # Tune dropout rate
    hp_dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
    model.add(Dropout(hp_dropout))

    # 2nd convolution layer
    model.add(Conv2D(128, (5,5), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(hp_dropout))

    # 3rd convolution layer
    model.add(Conv2D(512, (3,3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(hp_dropout))

    # Flatten and feed into dense layer
    model.add(Flatten())
    # Tune the number of units in the dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(Dense(hp_units))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(hp_dropout))

    # Output layer
    model.add(Dense(7, activation='softmax'))

    # Compile model
    # Tune learning rate
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=Adam(learning_rate=hp_learning_rate, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='./test',
    project_name='facial_expression_recognition'
)

# Stop training when validation loss stops improving
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(train_generator, epochs=20, validation_data=test_generator, callbacks=[stop_early])

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

# Print results
print(best_hps)

Trial 30 Complete [00h 03m 09s]
val_accuracy: 0.5495960116386414

Best val_accuracy So Far: 0.6064363121986389
Total elapsed time: 01h 01m 35s


In [9]:
print(best_hps.values)

{'dropout': 0.1, 'units': 416, 'learning_rate': 0.0001, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0023'}
